In [3]:
def process_seismic_data(tr_data_filt, sampling_rate, nperseg=256):
    """
    Process seismic data to extract velocity and frequency features in the same shape.
    
    Parameters:
    tr_data_filt : np.array
        Filtered time series data
    sampling_rate : float
        Sampling rate of the seismic data
    nperseg : int
        Length of each segment for spectrogram calculation
    
    Returns:
    X : np.array
        Feature matrix with shape (n_samples, n_features)
    feature_names : list
        Names of the features
    """
    from scipy import signal
    
    # Calculate spectrogram
    frequencies, times, spectrogram = signal.spectrogram(
        tr_data_filt, 
        fs=sampling_rate,
        nperseg=nperseg
    )
    
    # Reshape the data for training
    n_timepoints = spectrogram.shape[1]
    n_freq_bins = spectrogram.shape[0]
    
    # Create feature matrix
    X = np.zeros((n_timepoints, n_freq_bins + 1))
    
    # For each time point in the spectrogram
    for i in range(n_timepoints):
        # Add the velocity value
        start_idx = i * (nperseg // 2)
        end_idx = start_idx + nperseg
        if end_idx > len(tr_data_filt):
            end_idx = len(tr_data_filt)
        X[i, 0] = np.mean(tr_data_filt[start_idx:end_idx])
        
        # Add the frequency components
        X[i, 1:] = spectrogram[:, i]
    
    # Create feature names
    feature_names = ['velocity'] + [f'freq_{f:.2f}Hz' for f in frequencies]
    
    return X, feature_names, times, frequencies

# Example usage
X, feature_names, times, frequencies = process_seismic_data(
    tr_data_filt,
    tr_filt.stats.sampling_rate
)

# Print some information about the processed data
print(f"Shape of feature matrix: {X.shape}")
print(f"Number of features: {len(feature_names)}")
print(f"Feature names: {feature_names[:5]}...")  # First 5 features

# Optionally, create a DataFrame for easier handling
df = pd.DataFrame(X, columns=feature_names)
df['time'] = times

# Example visualization of the processed data
plt.figure(figsize=(12, 6))
plt.subplot(211)
plt.plot(times, df['velocity'])
plt.title('Velocity over Time')
plt.xlabel('Time (s)')
plt.ylabel('Velocity (m/s)')

plt.subplot(212)
plt.imshow(X[:, 1:].T, aspect='auto', origin='lower', 
           extent=[times[0], times[-1], frequencies[0], frequencies[-1]])
plt.colorbar(label='Power')
plt.title('Frequency Components over Time')
plt.xlabel('Time (s)')
plt.ylabel('Frequency (Hz)')
plt.tight_layout()
plt.show()

Processing MSEED files...


100%|██████████| 76/76 [00:33<00:00,  2.28it/s]


Saving processed data to processed_seismic_data_flattened.csv

Dataset Statistics:
Number of time points: 190597
Number of features: 133

Sample of feature names:
- filename
- arrival_time
- relative_arrival
- time
- freq_0.00Hz
- freq_0.03Hz
- freq_0.05Hz
- freq_0.08Hz
- freq_0.10Hz
- freq_0.13Hz
